In [14]:
import seml
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
df_experiments = seml.get_results('rgnn_global_attacks',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])


/nfs/homedirs/schmidtt/miniconda3/lib/python3.8/site-packages/seml/evaluation.py:48: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  parsed = pd.io.json.json_normalize(parsed, sep='.')


In [17]:
df_results = [
    pd.DataFrame(r) 
    for r in df_experiments['result.results']
]
for df_result, (_, df_experiment) in zip(df_results, df_experiments.iterrows()):
    df_result['dataset'] = df_experiment['config.dataset']
    df_result['attack'] = df_experiment['config.attack']
    df_result['seed'] = df_experiment['config.seed']
    df_result['batch_id'] = df_experiment['batch_id']
    df_result['loss_type'] = df_experiment['config.attack_params.loss_type']
    df_result['loss_type'] = df_result['loss_type'].fillna("CE")
    df_result['novel_loss'] = (
        #(df_experiment['config.attack_params.stop_optimizing_if_label_flipped'] == True)
         (df_experiment['config.attack_params.loss_type'] == 'MCE')
        | (df_experiment['config.attack_params.loss_type'] == 'tanhMargin')
    )

df_results = pd.concat(df_results, ignore_index=True)
df_results = df_results.sort_values('batch_id')
df_results = df_results.drop_duplicates([
    c for c in df_results.columns if c != 'batch_id' and c != 'accuracy'
], keep='last')

df_results

,label,epsilon,accuracy,dataset,attack,seed,batch_id,loss_type,novel_loss
0,Soft Medoid RPPRGo (T=0.2),0.00,0.797233,cora_ml,PGD,0,1,CE,False
2333,Soft Median GDC (T=0.5),0.25,0.754150,cora_ml,GreedyRBCD,5,1,MCE,True
2334,Soft Median GDC (T=0.2),0.25,0.756522,cora_ml,GreedyRBCD,5,1,MCE,True
2335,Jaccard GCN,0.25,0.145059,cora_ml,GreedyRBCD,5,1,MCE,True
2336,SVD GCN,0.25,0.139526,cora_ml,GreedyRBCD,5,1,MCE,True
...,...,...,...,...,...,...,...,...,...
1174,Vanilla GDC,0.00,0.830040,cora_ml,FGSM,1,1,CE,False
1175,Soft Medoid GDC (T=1.0),0.00,0.820158,cora_ml,FGSM,1,1,CE,False
1176,Soft Medoid GDC (T=0.5),0.00,0.813043,cora_ml,FGSM,1,1,CE,False
1165,Soft Medoid RPPRGo (T=0.5),0.25,0.710672,cora_ml,FGSM,1,1,CE,False


In [18]:
dataset_map = {
    'cora_ml': r'Cora ML', 
    'citeseer': r'Citeseer', 
}
dataset_order = [dataset_map[k] for k in dataset_map.keys()]

In [25]:
attack_map = {
    'PRBCD': r'PR-BCD',
    'DICE': r'DICE',
    'FGSM': r'FGSM',
    'PGD': r'PGD',
    'GANG': r'GANG',
    'GreedyRBCD': r'Greedy RBCD',
}
attack_order = [attack_map[k] for k in attack_map.keys()]

In [26]:
def transform_label(label: str):
    return label

In [27]:
def calc_mean_and_error(values: pd.Series, seeds: pd.Series, with_error=True, decimal_places: int = 3): 
    values, seeds = values.values, seeds.values
    seeds = seeds[~np.isnan(values)]
    values = values[~np.isnan(values)]

    idx = np.unique(seeds, return_index=True)[1]
    values = values[idx]
    
    if with_error:
        return rf'{np.mean(values):.{decimal_places}f} $\pm$ {np.std(values)/len(values):.{decimal_places}f}'
    else:
        return rf'{np.mean(values):.{decimal_places}f}'

In [28]:
architecture_c = r'Architecture'
dataset_c = r'   '
attack_c = r'Attack'
epsilons_c = r'Frac. edges (epsilon)'

epsilons = [0.05, 0.1, 0.25]
epsilon_marks = [r'\textit', r'\underline', r'\textbf']

In [32]:

df_cora = []
df_citeseer = []

for (dataset, attack, label, epsilon, loss_type), df_group in df_results.groupby(['dataset', 'attack', 'label', 'epsilon', "loss_type"]):
       if len(df_group.seed.unique()) != 5:
              print(f'For {dataset}-{attack}-{epsilon}-{loss_type} collected runs for seed {df_group.seed.tolist()}')

       accurcy = calc_mean_and_error(df_group.accuracy, df_group.seed, with_error=False)
       if dataset == "citeseer":
              df_citeseer.append({
                     dataset_c: dataset_map[dataset],
                     architecture_c: transform_label(label),
                     attack_c: attack_map[attack],
                     epsilons_c: epsilon,
                     "loss_type": loss_type,
                     'accuracy': accurcy
              })
       elif dataset == "cora_ml":
              df_cora.append({
                     dataset_c: dataset_map[dataset],
                     architecture_c: transform_label(label),
                     attack_c: attack_map[attack],
                     epsilons_c: epsilon,
                     "loss_type": loss_type,
                     'accuracy': accurcy
              })


df_cora = pd.DataFrame(df_cora)
df_citeseer = pd.DataFrame(df_citeseer)

For cora_ml-GreedyRBCD-0.0-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.01-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.05-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.1-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.25-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.0-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.01-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.05-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.1-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.25-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.0-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.01-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.05-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.1-MCE collected runs for seed [5, 42, 0, 1]
For cora_ml-GreedyRBCD-0.2

In [33]:

print(df_cora.to_markdown())

|     |         | Architecture               | Attack      |   Frac. edges (epsilon) | loss_type   |   accuracy |
|----:|:--------|:---------------------------|:------------|------------------------:|:------------|-----------:|
|   0 | Cora ML | Jaccard GCN                | DICE        |                    0    | CE          |      0.802 |
|   1 | Cora ML | Jaccard GCN                | DICE        |                    0.01 | CE          |      0.801 |
|   2 | Cora ML | Jaccard GCN                | DICE        |                    0.05 | CE          |      0.792 |
|   3 | Cora ML | Jaccard GCN                | DICE        |                    0.1  | CE          |      0.785 |
|   4 | Cora ML | Jaccard GCN                | DICE        |                    0.25 | CE          |      0.753 |
|   5 | Cora ML | RGCN                       | DICE        |                    0    | CE          |      0.8   |
|   6 | Cora ML | RGCN                       | DICE        |                    0.01 | C

In [31]:
df_cora

,,Architecture,Attack,Frac. edges (epsilon),accuracy
0,Cora ML,Jaccard GCN,DICE,0.00,0.802
1,Cora ML,Jaccard GCN,DICE,0.01,0.801
2,Cora ML,Jaccard GCN,DICE,0.05,0.792
3,Cora ML,Jaccard GCN,DICE,0.10,0.785
4,Cora ML,Jaccard GCN,DICE,0.25,0.753
...,...,...,...,...,...
625,Cora ML,Vanilla PPRGo,PGD,0.05,0.757
626,Cora ML,Vanilla PPRGo,PGD,0.10,0.738
627,Cora ML,Vanilla PPRGo,PGD,0.10,0.704
628,Cora ML,Vanilla PPRGo,PGD,0.25,0.643


In [14]:
df_citeseer

,,Architecture,Attack,Frac. edges (epsilon),accuracy
0,Citeseer,Jaccard GCN,PR-BCD,0.00,0.707
1,Citeseer,Jaccard GCN,PR-BCD,0.01,0.694
2,Citeseer,Jaccard GCN,PR-BCD,0.05,0.654
3,Citeseer,Jaccard GCN,PR-BCD,0.10,0.623
4,Citeseer,Jaccard GCN,PR-BCD,0.25,0.532
5,Citeseer,RGCN,PR-BCD,0.00,0.664
6,Citeseer,RGCN,PR-BCD,0.01,0.646
7,Citeseer,RGCN,PR-BCD,0.05,0.608
8,Citeseer,RGCN,PR-BCD,0.10,0.568
9,Citeseer,RGCN,PR-BCD,0.25,0.478


In [6]:
df_experiments

,_id,batch_id,slurm.experiments_per_job,slurm.sbatch_options.gres,slurm.sbatch_options.mem,slurm.sbatch_options.cpus-per-task,slurm.sbatch_options.time,slurm.sbatch_options.nodes,slurm.sbatch_options.job-name,slurm.sbatch_options.array,slurm.sbatch_options.output,slurm.array_id,slurm.task_id,config.overwrite,config.db_collection,config.dataset,config.attack,config.attack_params.do_synchronize,config.attack_params.epochs,config.attack_params.fine_tune_epochs,config.attack_params.keep_heuristic,config.attack_params.loss_type,config.attack_params.search_space_size,config.epsilons,config.surrogate_params.n_filters,config.surrogate_params.dropout,config.surrogate_params.train_params.lr,config.surrogate_params.train_params.weight_decay,config.surrogate_params.train_params.patience,config.surrogate_params.train_params.max_epochs,config.binary_attr,config.seed,config.artifact_dir,config.pert_adj_storage_type,config.pert_attr_storage_type,config.model_storage_type,config.device,config.display_steps,config.model_label,config.make_undirected,config.make_unweighted,config.data_device,config.data_dir,result.results
0,1,1,1,gpu:1,16G,4,0-08:00,1,rgnn_at_scale_attack_evasion_transfer_1,0-14,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6298148,0,1,kdd21_attack_evasion_transfer_prbcd,citeseer,PRBCD,True,400,100,WeightOnly,MCE,1000000,"[0, 0.01, 0.05, 0.1, 0.25]",64,0.5,0.01,0.001,200,300,False,0,cache,evasion_transfer_attack_adj,evasion_transfer_attack_attr,attack_citeseer,0,10,None,True,True,0,/nfs/students/schmidtt/datasets/,"[{'label': 'Vanilla PPRGo', 'epsilon': 0, 'acc..."
1,2,1,1,gpu:1,16G,4,0-08:00,1,rgnn_at_scale_attack_evasion_transfer_1,0-14,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6298148,1,2,kdd21_attack_evasion_transfer_prbcd,citeseer,PRBCD,True,400,100,WeightOnly,CE,1000000,"[0, 0.01, 0.05, 0.1, 0.25]",64,0.5,0.01,0.001,200,300,False,0,cache,evasion_transfer_attack_adj,evasion_transfer_attack_attr,attack_citeseer,0,10,None,True,True,0,/nfs/students/schmidtt/datasets/,"[{'label': 'Vanilla PPRGo', 'epsilon': 0, 'acc..."
2,3,1,1,gpu:1,16G,4,0-08:00,1,rgnn_at_scale_attack_evasion_transfer_1,0-14,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6298148,2,3,kdd21_attack_evasion_transfer_prbcd,citeseer,PRBCD,True,400,100,WeightOnly,tanhMargin,1000000,"[0, 0.01, 0.05, 0.1, 0.25]",64,0.5,0.01,0.001,200,300,False,0,cache,evasion_transfer_attack_adj,evasion_transfer_attack_attr,attack_citeseer,0,10,None,True,True,0,/nfs/students/schmidtt/datasets/,"[{'label': 'Vanilla PPRGo', 'epsilon': 0, 'acc..."
3,4,1,1,gpu:1,16G,4,0-08:00,1,rgnn_at_scale_attack_evasion_transfer_1,0-14,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6298148,3,4,kdd21_attack_evasion_transfer_prbcd,citeseer,PRBCD,True,400,100,WeightOnly,MCE,1000000,"[0, 0.01, 0.05, 0.1, 0.25]",64,0.5,0.01,0.001,200,300,False,1,cache,evasion_transfer_attack_adj,evasion_transfer_attack_attr,attack_citeseer,0,10,None,True,True,0,/nfs/students/schmidtt/datasets/,"[{'label': 'Vanilla PPRGo', 'epsilon': 0, 'acc..."
4,5,1,1,gpu:1,16G,4,0-08:00,1,rgnn_at_scale_attack_evasion_transfer_1,0-14,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6298148,4,5,kdd21_attack_evasion_transfer_prbcd,citeseer,PRBCD,True,400,100,WeightOnly,CE,1000000,"[0, 0.01, 0.05, 0.1, 0.25]",64,0.5,0.01,0.001,200,300,False,1,cache,evasion_transfer_attack_adj,evasion_transfer_attack_attr,attack_citeseer,0,10,None,True,True,0,/nfs/students/schmidtt/datasets/,"[{'label': 'Vanilla PPRGo', 'epsilon': 0, 'acc..."
5,6,1,1,gpu:1,16G,4,0-08:00,1,rgnn_at_scale_attack_evasion_transfer_1,0-14,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6298148,5,6,kdd21_attack_evasion_transfer_prbcd,citeseer,PRBCD,True,400,100,WeightOnly,tanhMargin,1000000,"[0, 0.01, 0.05, 0.1, 0.25]",64,0.5,0.01,0.001,200,300,False,1,cache,evasion_transfer_attack_adj,evasion_transfer_attack_attr,attack_citeseer,0,10,None,True,True,0,/nfs/students/schmidtt/datasets/,"[{'label': 'Vanilla PPRGo', 'epsilon': 0, 'acc..."
6,7,1,1,gpu:1,16G,4,0-08:00,1,